## Step 1 : Model choice and loading

In [ ]:
from transformers import pipeline

In [ ]:
pipe = pipeline("text-generation", model="openai-community/gpt2", return_full_text=False)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cpu


In [ ]:
prompt = "Once upon a time in a small village"
results = pipe(prompt, max_length=50, num_return_sequences=2)
for i, r in enumerate(results):
    print(f"Output {i}:", r["generated_text"])

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=256) and `max_length`(=50) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Output 0: Once upon a time in a small village, a pair of knights came on a horse and cut off the eye of one of the nobles. The knight fell to his knees, his face red in pain. However, a knight's life had been spared. He was the one to save the life of the knight.

But it was not the knight's life that was spared. The knight's life would be completely lost.

The knight's life was in the hands of the people of the village. They had been sent to protect the village from the invading army. He would die there. His life would be forever lost.

The village did not know that he had been sent here. It was a city, but there was no one here to protect it. The people of the village were so scared of the people of the village that they did not dare to enter the village and do something about it.

However, the people of the village were so frightened that they refused to let them enter.

The village was a city. It was a place that was created by the people of the village. They had to live in fear of

#### the model Loading is working !!

## Step 2 : Selecting the downstream task and data

- the idea behind that is to use the model and give it a simple passage and tell it to generate questions and answers in order to run the **SEAL** method on it

In [ ]:

context = """A large language model (LLM) is a language model with a large number of parameters (generally more than a billion).

These are deep neural networks trained on large amounts of unlabeled text using self-supervised learning. LLMs appeared around 2017 and have been used to implement conversational agents.

Instead of being trained for a specific task such as sentiment analysis, named entity recognition, or mathematical reasoning, they can accomplish a wide range of tasks. They are first pre-trained to predict a likely continuation for a given input. The quality of generated content tends to increase with the number of parameters, the size and quality of training data, and the amount of compute used to train the model. Large language models are then most often fine-tuned to adopt the role of a conversational assistant and to be “helpful, honest, and harmless.”

Language models with a large number of parameters can capture much of the syntax and semantics of human language. This enables them to reproduce substantial general world knowledge, with memorization of many facts during training.

Before the success of large language models, NLP research mainly focused on supervised learning of specialized models for specific tasks."""

qa_prompt = f"""You are an expert teacher in natural language processing and your task is to generate **question-and-answer pairs** that test a reader’s understanding of a short technical passage.

**Instructions:**
1. Use the passage provided below.
2. Generate **5 distinct question-and-answer pairs**.
3. Each question should be clear, concise, and focus on a key concept from the passage.
4. Each answer should be correct, complete, and directly based on the passage (no outside knowledge).
5. Format your output exactly as follows:

1) Question: <question_1>
   Answer: <answer_1>
2) Question: <question_2>
   Answer: <answer_2>
…
5) Question: <question_5>
   Answer: <answer_5>

**Passage:**
{context}
"""

results = pipe(qa_prompt, max_length=300, num_return_sequences=1)
print(results[0]["generated_text"])


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=256) and `max_length`(=300) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


 recognition, or the like, we have trained for a large number of tasks. We want to look for ways to capture complex behavior within the language structure. 

Below are some examples of how to do this:

Question:

In the sentence, you mean

Question: <answer_2>

The answer: <answer_2> 

5) Question: <question_3> 

You can then use the following training algorithm to predict:

1) Question: <question_1> [x <x]]

Or

Question: <question_2> [x <x]]

Or

Question: <question_3> [x <x]]

Or

Question: <question_4> [x <x]]

So I can then give a prediction about:

Question: <answer_2> [x <x]]

Or

Question: <answer_3> [x <x]]

Which is a good result if we are able to predict:

1) Question: <answer_1> [x <x]]

Or

Question: <answer


- I tried to generate the initial context Q.A using the same model but based on it's capabilities it couldn't generate them successfuly .

In [ ]:
## baseline of the model before doing the SEAL method
import pandas as pd

df = pd.read_json('questionAndanswers.json')
df.head()

,question,answer
0,What is a large language model (LLM)?,A language model with a large number of parame...
1,Around when did LLMs appear?,They appeared around 2017.
2,What kind of tasks can LLMs accomplish?,A wide range of tasks (not just sentiment anal...
3,How are LLMs pre-trained?,They are pretrained to predict a likely contin...
4,What factors improve the quality of generated ...,"Larger number of parameters, bigger and higher..."


In [ ]:
for i in df['question']:
    results = pipe(i, max_length=100, num_return_sequences=1)
    print(f"Question: {i}")
    print(f"Answer: {results[0]['generated_text']}")
    print("-----\n")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=256) and `max_length`(=100) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=256) and `max_length`(=100) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Question: What is a large language model (LLM)?
Answer: 

As I've discussed in my previous article, this is an approach to translating a single language into a multi-language model. An example of this is the language of the English language and its grammar.

A typical language model can be very simple (like "English" or "Spanish") or very complex (like "Spanish") using many different language features. For example, English has a huge number of features that make it a great translation language.

In a typical language model, the language itself is represented by a system of language features that represent a number of possible languages and their grammar. In a multi-language model, the language features represent the number of possible languages represented by different languages. In English, the grammar is represented by the same system that we might find at the top of the English language pyramid.

The number of possible languages in a multi-language model is not just a function of th

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=256) and `max_length`(=100) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Question: Around when did LLMs appear?
Answer: 

The reason they appeared was because in the early 1990s, there was a very concerted effort by the government to get the government to allow it to carry out some sort of review of the LLMs. The government was in talks with the LLM industry to see if they would come up with some way of improving the system.

When you look at the LLMs, they're quite simple. You put them in a box and they're placed in a box. It's a bit like adding a pair of glasses to your glasses – no need to put them in a box.

What were you doing to make them more practical?

We were looking for a way to make them more durable. We were looking for ways to improve them in a very practical way. We liked the idea that there would be a way of making them less expensive and easier to use. The LLMs are actually a very simple device. The most effective way to do this is to use them as a little stand and a little table. They're very practical in the sense that they are very easy 

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=256) and `max_length`(=100) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Question: What kind of tasks can LLMs accomplish?
Answer: 

The most common tasks involve doing things quickly and efficiently. There are multiple types of tasks, such as:

Writing code

Applying code to database

Recognizing user actions

Designing system-wide services

Writing applications

Writing code in other languages

Writing code in C#

And the C# language is where we are now. LLMs are extremely powerful, so it's important to know how to use them.

So how do you use LLMs to write a program?

To begin, first define a few variables that you want to use.

void foo(string string) { printf("Hello, world!

"); return; }

Next, define a class called foo{};

class Foo { public int x; public int y; public int x; }

Finally, define a class called foo{BEGINNING};

class Foo { public int x; public int y; public int x; }

We'll use the following code to write the program:

void foo(string string) { printf("Hello, world!

"); return; }

In our program, we'll write the following code:
-----



KeyboardInterrupt: 

In [ ]:
Baseline_answers = []

for i in df['question']:
    results = pipe(i, max_length=20, num_return_sequences=1)
    Baseline_answers.append(results[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=256) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=256) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=256) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Setting `pad_token_id` to `eos_token_

In [ ]:
import re

def normalize(text):
    text = text.lower().strip()
    text = re.sub(r"[^\w\s]", "", text)   # remove punctuation
    text = re.sub(r"\s+", " ", text)
    return text

def f1_score(pred, truth):
    pred_tokens = normalize(pred).split()
    truth_tokens = normalize(truth).split()
    if len(pred_tokens) == 0 or len(truth_tokens) == 0:
        return 0.0
    common = set(pred_tokens) & set(truth_tokens)
    if not common:
        return 0.0
    prec = len(common) / len(pred_tokens)
    rec  = len(common) / len(truth_tokens)
    return 2 * (prec * rec) / (prec + rec)

def exact_match(pred, truth):
    return 1 if normalize(pred) == normalize(truth) else 0



ems = []
f1s = []
for pred, item in zip(Baseline_answers, df.to_dict(orient="records")):
    ems.append(exact_match(pred, item["answer"]))
    f1s.append(f1_score(pred, item["answer"]))

print("Exact Match avg:", sum(ems) / len(ems))
print("F1 avg:", sum(f1s) / len(f1s))


Exact Match avg: 0.0
F1 avg: 0.044996048955676195


####  Based on the low performace of GPT 2 ,I will follow the Student-Teacher method , where i will use a better model eg GPT-5 to generate the self-edits and then implement the **algorithm** on the smaller model   

In [ ]:


## Step 3: Fine-tune with Self-Edits (LoRA)

import json
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments, DataCollatorForLanguageModeling
from peft import LoraConfig, get_peft_model, TaskType
from datasets import Dataset

# --- Load self-edits ---
with open("self_edits.json", "r", encoding="utf-8") as f:
    self_edits = json.load(f)

# --- Prepare training data ---
# Format: "Question: ... Answer: ..." for each synthetic example
train_texts = []
for edit in self_edits:
    # Extract synthetic example (could be statement or Q/A pair)
    text = edit["synthetic_example"]
    # Ensure it ends with a period for proper tokenization
    if not text.endswith(('.', '!', '?')):
        text += "."
    train_texts.append(text)

# Create HF Dataset
train_dataset = Dataset.from_dict({"text": train_texts})

# --- Load base model and tokenizer ---
model_name = "openai-community/gpt2"  # Same as your baseline
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token  # GPT-2 needs this

base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float32,  # CPU-friendly (avoid fp16 on CPU)
    device_map="cpu"
)

# --- Apply LoRA (parameter-efficient fine-tuning) ---
lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=8,               # Low rank for CPU
    lora_alpha=16,
    lora_dropout=0.1,
    target_modules=["c_attn"],  # GPT-2 attention layers
)
model = get_peft_model(base_model, lora_config)
model.print_trainable_parameters()  # Should be ~0.3% of total params

# --- Tokenize dataset ---
def tokenize_function(examples):
    return tokenizer(
        examples["text"],
        truncation=True,
        max_length=128,  # Keep short for speed
        padding="max_length"
    )

tokenized_dataset = train_dataset.map(tokenize_function, batched=True, remove_columns=["text"])

# --- Training arguments (CPU-optimized) ---
training_args = TrainingArguments(
    output_dir="./lora_finetuned_gpt2",
    overwrite_output_dir=True,
    num_train_epochs=2,           # Use directive's epochs if you want (2-5 typical)
    per_device_train_batch_size=2, # Very small for CPU
    gradient_accumulation_steps=4, # Simulate batch_size=8
    learning_rate=2e-5,            # Median directive value
    logging_steps=5,
    save_steps=50,
    save_total_limit=1,
    fp16=False,                    # CPU doesn't support fp16
    report_to="none",              # Disable wandb
    dataloader_num_workers=0,      # CPU-safe
)

# --- Data collator ---
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False  # Causal LM (not masked LM)
)

# --- Trainer ---
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=data_collator,
)

# --- Fine-tune ---
print("Starting LoRA fine-tuning...")
trainer.train()

# --- Save LoRA adapter ---
model.save_pretrained("./lora_adapter")
tokenizer.save_pretrained("./lora_adapter")
print("✅ LoRA adapter saved to ./lora_adapter")



`torch_dtype` is deprecated! Use `dtype` instead!
/usr/local/lib/python3.12/dist-packages/peft/tuners/lora/layer.py:2174: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


trainable params: 294,912 || all params: 124,734,720 || trainable%: 0.2364


Map:   0%|          | 0/20 [00:00<?, ? examples/s]

The model is already on multiple devices. Skipping the move to device specified in `args`.


Starting LoRA fine-tuning...


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
`loss_type=None` was set in the config but it is unrecognized. Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
5,4.830900


✅ LoRA adapter saved to ./lora_adapter


* the model Now is finetuned one round on the new self edits , wee need to test it agaisnt the old evaluation questions

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel
import re


base_model_name = "openai-community/gpt2"
tokenizer = AutoTokenizer.from_pretrained("./lora_adapter")
tokenizer.pad_token = tokenizer.eos_token

base_model = AutoModelForCausalLM.from_pretrained(base_model_name)
model = PeftModel.from_pretrained(base_model, "./lora_adapter")
model = model.to("cpu")


eval_data = df.to_dict(orient="records")


def generate_answer(question):
    prompt = f"Question: {question}\nAnswer:"
    inputs = tokenizer(prompt, return_tensors="pt")
    outputs = model.generate(**inputs, max_new_tokens=100, do_sample=False)
    generated = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # remove the prompt part
    answer = generated.split("Answer:")[-1].strip()
    return answer



ems, f1s = [], []
for item in eval_data:
    pred = generate_answer(item["question"])
    ems.append(exact_match(pred, item["answer"]))
    f1s.append(f1_score(pred, item["answer"]))

print("Exact Match avg:", sum(ems)/len(ems))
print("F1 avg:", sum(f1s)/len(f1s))


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end gene

Exact Match avg: 0.0
F1 avg: 0.05495829582714453


* the next step now is to select the best self edits to keep them for the next round , this is kind of semulating the reinforecement learning step described in the paper .

In [ ]:



from transformers import AutoModelForCausalLM
from peft import PeftModel
import torch

def generate_answer(model, tokenizer, question):
    """Generate a short answer for a given question using the model."""
    prompt = f"Question: {question}\nAnswer:"
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=256)
    inputs = {k: v.to(model.device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=80,
            do_sample=False,
            pad_token_id=tokenizer.eos_token_id,
        )

    generated = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # Keep only the text after "Answer:"
    answer = generated.split("Answer:")[-1].strip()
    return answer

def fine_tune_one_edit(model, edit, tokenizer):
    """Ensure this function properly computes loss with gradients"""
    optimizer = torch.optim.AdamW(
        [p for p in model.parameters() if p.requires_grad],
        lr=1e-4
    )

    # Prepare input
    inputs = tokenizer(
        edit["synthetic_example"],
        return_tensors="pt",
        truncation=True,
        max_length=512
    )

    for key in inputs:
        inputs[key] = inputs[key].to(model.device)

    # Forward pass - ensure model returns loss
    outputs = model(**inputs, labels=inputs["input_ids"])
    loss = outputs.loss  # This should have requires_grad=True

    # Verify loss has gradient
    if not loss.requires_grad:
        raise RuntimeError("Loss does not require grad - check model outputs")

    # Backward pass
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    return loss.item()


def evaluate_model(model, eval_data):
    """
    Evaluate model on a list of {question, answer} pairs.
    Returns (average_f1, average_em).
    """
    model.eval()
    ems, f1s = [], []

    for item in eval_data:
        pred = generate_answer(model, tokenizer, item["question"])
        ems.append(exact_match(pred, item["answer"]))
        f1s.append(f1_score(pred, item["answer"]))

    avg_f1 = sum(f1s) / len(f1s)
    avg_em = sum(ems) / len(ems)
    return avg_f1, avg_em

results = []
validation_set = eval_data  # your original 20 QA pairs
initial_f1 = 0.04499    # the F1 before trying this edit

def load_student_model():
    base = AutoModelForCausalLM.from_pretrained(base_model_name)
    student = PeftModel.from_pretrained(base, "./lora_adapter")
    student = student.to("cpu")
    student.train()  # make sure requires_grad = True on adapter params
    return student

for i, edit in enumerate(self_edits):
    print(f"Testing self-edit {i+1}/{len(self_edits)}")

    # reload fresh LoRA student each time
    base_model = AutoModelForCausalLM.from_pretrained(base_model_name)
    model_copy = PeftModel.from_pretrained(base_model, "./lora_adapter").to("cpu")

    # Critical: Enable gradient computation and set model to train mode
    model_copy.train()

    # Verify that LoRA parameters require gradients
    for name, param in model_copy.named_parameters():
        if 'lora' in name.lower():
            param.requires_grad = True

    # Enable gradient tracking globally
    torch.set_grad_enabled(True)

    # Fine-tune with the edit
    fine_tune_one_edit(model_copy, edit, tokenizer)

    # Evaluate model_copy
    new_f1, new_em = evaluate_model(model_copy, eval_data)
    delta_f1 = new_f1 - initial_f1

    results.append({
        "edit_index": i,
        "synthetic_example": edit["synthetic_example"],
        "directive": edit["directive"],
        "delta_f1": delta_f1
    })

    # Clean up to free memory
    del model_copy
    del base_model
    torch.cuda.empty_cache() if torch.cuda.is_available() else None

# Rank edits by improvement
sorted_edits = sorted(results, key=lambda x: x["delta_f1"], reverse=True)

# Select top edits
best_edits = [e for e in sorted_edits if e["delta_f1"] > 0]

print("\nTop-performing self-edits:")
for e in best_edits:
    print(f"ΔF1 = {e['delta_f1']:.3f} | Example: {e['synthetic_example'][:60]}")

Testing self-edit 1/20
Testing self-edit 2/20
Testing self-edit 3/20
Testing self-edit 4/20
Testing self-edit 5/20
Testing self-edit 6/20
Testing self-edit 7/20
Testing self-edit 8/20
Testing self-edit 9/20
Testing self-edit 10/20
Testing self-edit 11/20
Testing self-edit 12/20
Testing self-edit 13/20
Testing self-edit 14/20
Testing self-edit 15/20
Testing self-edit 16/20
Testing self-edit 17/20
Testing self-edit 18/20
Testing self-edit 19/20
Testing self-edit 20/20

Top-performing self-edits:
ΔF1 = 0.021 | Example: A large language model (LLM) is defined as a neural network 
ΔF1 = 0.021 | Example: Question: When were large language models first introduced? 
ΔF1 = 0.021 | Example: LLMs demonstrate remarkable versatility, capable of performi
ΔF1 = 0.021 | Example: Question: What is the fundamental objective during LLM pre-t
ΔF1 = 0.021 | Example: The quality of content generated by large language models im
ΔF1 = 0.021 | Example: Question: What typically follows the pre-training phase in